In [ ]:
import dash
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output,State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
#from datetime import datetime
#from collections import deque
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
import base64
import time
import random

/Users/marielambart/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])

#Data :
merged_df = pd.read_csv('ML_df_advanced.csv')
genre_df = pd.get_dummies(merged_df['genre'].str.split(',',expand=True).replace({None:np.nan}).stack()).groupby(level=0).max()
countries_df = pd.get_dummies(merged_df['country'].str.split(',',expand=True).replace({None:np.nan}).stack()).groupby(level=0).max()
directors_df = pd.get_dummies(merged_df['director'].str.split(',',expand=True).replace({None:np.nan}).stack()).groupby(level=0).max()
actors_df = pd.get_dummies(merged_df['actor'].str.split(',',expand=True).replace({None:np.nan}).stack()).groupby(level=0).max()
ML_model_df = pd.merge(merged_df,genre_df,left_index=True,right_index=True)
ML_model_df = pd.merge(ML_model_df,countries_df,left_index=True,right_index=True)
ML_model_df = pd.merge(ML_model_df,directors_df,left_index=True,right_index=True)
ML_model_df = pd.merge(ML_model_df,actors_df,left_index=True,right_index=True)

# Visuel carroussel :

img_1 = 'carroussel_1.png'
test_base64_img_1 = base64.b64encode(open(img_1, 'rb').read()).decode('ascii')
img_2 = 'carroussel_2.png'
test_base64_img_2 = base64.b64encode(open(img_2, 'rb').read()).decode('ascii')
img_3 = 'carroussel_3.png'
test_base64_img_3 = base64.b64encode(open(img_3, 'rb').read()).decode('ascii')
img_4 = 'carroussel_4.png'
test_base64_img_4 = base64.b64encode(open(img_4, 'rb').read()).decode('ascii')
img_5 = 'carroussel_5.png'
test_base64_img_5 = base64.b64encode(open(img_5, 'rb').read()).decode('ascii')

# Visuel titre :
titre = 'titre.png'
test_base64_titre = base64.b64encode(open(titre, 'rb').read()).decode('ascii')



# Fonction pour récupérer l'affichage des films sous forme de liste : 
def display_movie_poster(poster, movie_title,synopsis,rating,director,actor):
    results_list = []
    movie_id=0
    for i in range(len(poster)):
        movie_id+=1
        results_list.append(
            html.Div([
                dbc.Row([
                    dbc.Col(width=1),
                    
                    dbc.Col(
                        html.Img(src=poster[i],
                                 className = 'movie-poster',
                                 style={'align':'center','width':'150px'},
                                width=1),
                    ),
                    dbc.Col(
                        html.H4(movie_title[i]),
                                className='movie-title text',
                                style = {'textAlign': 'center','color':'white'},
                                width=1
                    ),
                    dbc.Col(
                        html.H6(synopsis[i]),
                        className ='movie_synopsis',
                        style={'textAlign': 'center','color':'white', 'textAlign': 'justify'},
                        width=4   
                    ),
                    
                    dbc.Col(
                        html.H6(str(rating[i])+ " / 10"),
                                className="rating_movie",
                                style = {'textAlign': 'center','color':'white'},
                                width=1
                    
                    ),
                    
                    dbc.Col(
                        html.H6(director[i]),
                        className ='movie_year',
                        style={'textAlign': 'center','color':'white','vertical-Align':'middle'},
                        width=1
                    ),
                    dbc.Col(
                        html.H6(actor[i]),
                        className="movie_averagerating",
                        style={'textAlign': 'center','color':'white'},
                        width=1
                    ),
                    dbc.Col(width=1)                  
                ],justify="center",style={'marginTop':'70px'},align="center"),
            ],id=f"movie-box-{movie_id}",className='movie-box')
        )
    return results_list

# Affichage inital des 10 films les plus populaires et les mieux notés :
pop_movies = display_movie_poster([i for i in merged_df.sort_values(by=['numvotes','averagerating'], ascending=[False,False]).head(10)['poster_url']], \
                                  [i for i in merged_df.sort_values(by=['numvotes','averagerating'], ascending=[False,False]).head(10)['title']], \
                                  [i for i in merged_df.sort_values(by=['numvotes','averagerating'], ascending=[False,False]).head(10)['synopsis']],
                                  [i for i in merged_df.sort_values(by=['numvotes','averagerating'], ascending=[False,False]).head(10)['averagerating']],
                                  [i for i in merged_df.sort_values(by=['numvotes','averagerating'], ascending=[False,False]).head(10)['director']],
                                  [i for i in merged_df.sort_values(by=['numvotes','averagerating'], ascending=[False,False]).head(10)['actor']]                      
                                 )

# App :
app.layout = html.Div(children=[
    dbc.Container(
        dbc.Row([
            
            dbc.Col(width=3),
            
            dbc.Col([
                dbc.Carousel(
                    items = [
                        {'src':'data:image/png;base64,{}'.format(test_base64_img_1)},
                        {"src":'data:image/png;base64,{}'.format(test_base64_img_2)},
                        {"src":'data:image/png;base64,{}'.format(test_base64_img_3)},
                        {"src":'data:image/png;base64,{}'.format(test_base64_img_4)},
                        {"src":'data:image/png;base64,{}'.format(test_base64_img_5)} 
                    ],
                    controls=False,
                    indicators=False,
                    interval=2000,
                    ride='carousel',
                    style={'marginTop': '10px','width':'900px'}
                ),
            ]),
            
            dbc.Col(width=3),
            
            
            
        ],justify="center"),
    fluid=True),
    
    dbc.Container(
        dbc.Row([
            dbc.Col(width=3),
            
            dbc.Col(
                html.Img(src='data:image/png;base64,{}'.format(test_base64_titre),
                         className = 'title-app',
                         style={'marginTop':'50px','width':'700px'})    
            ),
            
            dbc.Col(width=4),
            
        ]),
    
    ),
    
    dbc.Container(
        dbc.Row([
        
            dbc.Col(width=3),

            dbc.Col([
                    dbc.Label("Enter the title of your movie",color='white'),
                    dbc.Input(id="user-search",placeholder="Type here and press Enter...", type="text")                   
                ],width=6),
            
            #dbc.Col([
                #dbc.Button("Submit",id="enter-search",color="danger",outline=True),
                
            #],style={'marginTop':'32px'}),

            dbc.Col(width=3),
        ],style={'marginTop':'50px'}),
    ),
    
    dcc.Loading(
        dbc.Container(
            pop_movies,
            #html.Div(id="movies-reco-display"),

        fluid=True,id="pop-movies-container"),
    id='loading-spinner', type='circle'),
    
    html.Div(id="movies-reco"), # Relatif à la recommandation
    
])



# 1er callback qui sert à stocker reco_movies
@app.callback(
    Output(component_id="movies-reco",component_property="children"),
    #Input(component_id="enter-search",component_property="n_clicks"),
    Input(component_id="user-search",component_property="n_submit"),
    State(component_id='user-search',component_property="value"),
    prevent_initial_call = True
)
def ML_reco(enter_search,user_search):  
    if enter_search:
        if user_search is not None:
            user_search = user_search.strip()
            if user_search not in (''):
                y = ML_model_df[['title']]
                X = ML_model_df.iloc[:,13:]
                n_neighbors = 10
                model = NearestNeighbors(n_neighbors=n_neighbors)
                model.fit(X)
                liste_movies = ML_model_df["title"].to_list()
                title_movie = process.extractOne(user_search,liste_movies)[0]  
                X_title_movie = ML_model_df[ML_model_df['title'] == title_movie].iloc[:,13:]
                results = model.kneighbors(X_title_movie)
                reco_df = merged_df.loc[[i for i in results[1][0,:]],['startyear','title','genre','synopsis','director','actor','country','runtimeminutes','isadult','averagerating','numvotes','poster_url']].fillna('unknown')

                # Dataframe en sorti du modèle du ML :
                #reco_df = ML_reco(user_search) #user_search = film taper dans la barre de recherche (id=user_search)


                # Affichage des 10 films recommandés à partir du film taper dans la barre de recherche :

                reco_movies = display_movie_poster([i for i in reco_df.head(10)['poster_url']], \
                                                   [i for i in reco_df.head(10)['title']], \
                                                   [i for i in reco_df.head(10)['synopsis']],
                                                   [i for i in reco_df.head(10)['averagerating']],
                                                   [i for i in reco_df.head(10)['director']],
                                                   [i for i in reco_df.head(10)['actor']]                      
                                                    )
                #print(user_search)
                #print(reco_movies)
                return reco_movies
            else:
                raise PreventUpdate
        else:
            raise PreventUpdate

@app.callback(Output(component_id="pop-movies-container",component_property="style"),
    Output(component_id="pop-movies-container",component_property="children"),         
    #Input(component_id='enter-search',component_property="n_clicks"),
    Input(component_id='user-search',component_property="n_submit"),
    State(component_id='user-search',component_property="value"),
    State(component_id="movies-reco",component_property="children"),
    prevent_initial_call = True
)
def display_container(enter_search, user_search, reco_movie):
    if enter_search:
        if user_search is not None:
            user_search = user_search.strip()
            if user_search not in (''):
                return {"display":"none"}, reco_movie
            else:
                raise PreventUpdate
        else:
            raise PreventUpdate
        


# Affichage de l'app :   
app.run_server(mode='external') 

Dash app running on http://127.0.0.1:8050/
